## This is the only cell you need to change ##

### Required packages ###
- numpy
- pandas

### Possible filter file ###
- Default: Set_filter_file.xlsx
- Any other filter file name as long as file exists in main directory
  
### Possible file formats ###
- CSV
- excel

### Possible output formats ###
- long format where each line contains one value
- wide table-style format where values for different years (and in case of trade parameters regions) are grouped per line

### Possible processing options ###
- both creates the dataset for parameters and timeseries
- parameters_only for only regular parameters (saves time since timeseries, especially with excel, takes a lot of time)
- timeseries_only for only timeseries

### Possible scenario options ###
- Any scenario name. The script will search for input data for the specified this scenario and overwrite the default values.

In [9]:
# Options regarding output format and processing
settings_file = 'Set_filter_file.xlsx' # Specify which filter file should be used
output_file_format = 'excel'  # Change this to 'csv' or 'excel' depending on your needs
output_format = 'long' # Change to 'wide' or 'long' depending on your needs
processing_option = 'parameters_only' # Change to 'parameters_only' if you don't want the timeseries to created (saves time) or 'timeseries_only' for only timeseries or 'both' 
scenario_option = 'Europe_EnVis_Trinity' # Specify your scenario here 

## Run this cell to execute the script ##
### You don't need to change anything in here ### 

In [10]:
from functions.function_import import master_function

# Call the function
master_function(settings_file,output_file_format, output_format, processing_option, scenario_option)


C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_CapacityToActivityUnit.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_EmissionPenaltyTagTech.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_ReserveMarginTagFuel.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_ReserveMarginTagTechnology.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_RETagFuel.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_RETagTechnology.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_TagDemandFuelToSector.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_TagElectricTechnology.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_TagFuelToSubsets.csv
C:\Users\nim\Documents\GENeSYS_MOD.data\Data\Parameters\00_Sets&Tags\Par_TagTechnologyToModalType.csv
C:\Users\nim\Documents\G